In [1]:
import os
from getpass import getpass
os.environ["TAVILY_API_KEY"] = getpass()


 ········


In [2]:
# Import relevant functionality
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [4]:
search.invoke("what is the weather in Diegem?")

[{'title': 'Weather in Diegem',
  'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Diegem', 'region': '', 'country': 'Belgium', 'lat': 50.9, 'lon': 4.4333, 'tz_id': 'Europe/Brussels', 'localtime_epoch': 1743001930, 'localtime': '2025-03-26 16:12'}, 'current': {'last_updated_epoch': 1743001200, 'last_updated': '2025-03-26 16:00', 'temp_c': 11.2, 'temp_f': 52.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 18, 'wind_dir': 'NNE', 'pressure_mb': 1024.0, 'pressure_in': 30.24, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 75, 'feelslike_c': 9.5, 'feelslike_f': 49.1, 'windchill_c': 13.7, 'windchill_f': 56.6, 'heatindex_c': 14.6, 'heatindex_f': 58.2, 'dewpoint_c': 3.6, 'dewpoint_f': 38.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.5, 'gust_mph': 11.1, 'gust_kph': 17.9}}",
  'score': 0.8681484},
 {'title': '10-Day Weather Forec

In [8]:
config = {"configurable" : {"thread_id" :"abc234"}}
# for step in agent_executor.stream(
#     {"messages":[HumanMessage(content="Hi im Pavel! and i live in belgium, ghent!")]},
#     config,
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()

In [9]:
for step in agent_executor.stream(
    {"messages" : [HumanMessage(content="whats the weather were i live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather were i live?
================================== Ai Message ==================================

To provide you with the current weather, I would need to know your location. Could you please specify the city or region you are in?


In [12]:
search.invoke("What is the weather in Ghent?")

[{'title': 'Ghent 30-Day Forecast - Province East Flanders - World-Weather.info',
  'url': 'https://world-weather.info/forecast/belgium/gent/month/',
  'content': '30-Day weather forecast in Ghent for March, April 2025 is based on analysis ... Monday, March 24. +45°. Day. +59°. Few clouds. Tuesday, March 25. +43°. Day. +',
  'score': 0.9302622},
 {'title': 'Ghent weather in March 2025 ⋆ Ghent temperature in March ...',
  'url': 'https://www.meteoprog.com/weather/Ghent/month/march/',
  'content': 'Ghent (Belgium) weather in March 2025 ☀️ Accurate weather forecast for Ghent ... 24 Mar. +13°+4°. 25 Mar. +12°+5°. 26 Mar. +11°+4°. 27 Mar. +12°+4°. 28 Mar. +13°+5',
  'score': 0.92731744}]

## Recreating this functionality using LangChain
### Using Language Models

In [10]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini",model_provider="openai")


In [11]:
model.invoke("Hi, I'm Pavel, how are you?")

AIMessage(content="Hi Pavel! I'm just a program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 16, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bbfba58e46', 'id': 'chatcmpl-BFMiEXQfDpqYc9cAEGYEyYJDakFsX', 'finish_reason': 'stop', 'logprobs': None}, id='run-eea418ac-da5a-4a16-9cb3-54b841ba21ed-0', usage_metadata={'input_tokens': 16, 'output_tokens': 35, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain_core.messages import HumanMessage
HumanMessage(content="Hi!")

HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={})

In [13]:
response = model.invoke([HumanMessage(content="Hi, how are you?")])
response.content

'Hello! I’m just a program, so I don’t have feelings, but I’m here and ready to help you. How can I assist you today?'

In [14]:
model_with_tools = model.bind_tools(tools)

A response of this new model has attributes `content` and `tool_calls`

In [15]:
response = model_with_tools.invoke([HumanMessage(content="What is the capital of France?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: The capital of France is Paris.
ToolCalls: []


In [16]:
response = model_with_tools.invoke([HumanMessage(content="What is the weather in Sf?")])

In [17]:
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather San Francisco'}, 'id': 'call_NPSo3DbANfhKDRReNMHzYwbc', 'type': 'tool_call'}]


In [18]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

In [ ]:
response = agent_executor.invoke({"messages" : [HumanMessage(content="what's the weather in Ghent?")]})

In [27]:
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

what's the weather in Ghent?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_T5mVXuxMy9mtD22Y4VdSsMEo)
 Call ID: call_T5mVXuxMy9mtD22Y4VdSsMEo
  Args:
    query: current weather in Ghent
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Ghent, East Flanders, Belgium Current Weather | AccuWeather", "url": "https://www.accuweather.com/en/be/ghent/30438/current-weather/30438", "content": "Current Weather. 3:48 AM. 40°F. Mostly clear.", "score": 0.73596483}, {"title": "Ghent, East Flanders, Belgium Weather Forecast - AccuWeather", "url": "https://www.accuweather.com/en/be/ghent/30438/weather-forecast/30438", "content": "Current Weather. 4:42 AM. 44°F. Partly cloudy. RealFeel® 41° · Looking Ahead. Rain Sunday afternoon. Ghent Weather Radar & Maps. Ghent We

In [28]:
for step in agent_executor.stream(
    {"messages" : HumanMessage(content="What is there to do in Ghent this weekend?")},
    stream_mode= "values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is there to do in Ghent this weekend?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_poJrzFhkTLEev41Cqxo7Ozvj)
 Call ID: call_poJrzFhkTLEev41Cqxo7Ozvj
  Args:
    query: things to do in Ghent this weekend
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Things to Do in Ghent This Weekend - Eventbrite", "url": "https://www.eventbrite.com/d/belgium--gent/events--this-weekend/", "content": "Ghent Business: Top 7 Deadly Mistakes That will DESTROY Your New Business\n\nFriday at 7:00 PM + 2 more\n\nGhent\n\nThings to do around Ghent\n\nTrends in Ghent\n\n1. Comedy shows\n\n2. Rave party\n\n3. Techno music\n\n4. Afterwork party\n\n5. Drag\n\n6. Gay party\n\n7. Cabaret cuberdon\n\n8. Techno\n\n9. Stand up comedy\n\n10. Rave\n\n11. House music\n\n12. Ha

### Streaming tokens

In [19]:
for step, metadata in agent_executor.stream(
    {"messages" : [HumanMessage(content="What is there to do in Lille at 11th and 12th of April?")]},
    stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

During| your| visit| to| Lille| on| April| |11|th| and| |12|th|,| |202|4|,| you| can| look| forward| to| a| variety| of| activities| and| events|.| Here| are| some| highlights|:

|1|.| **|C|ultural| Events|**|:| Lille| is| set| to| host| numerous| cultural| events| in| April|,| including| exhibitions| and| festivities| related| to| the| upcoming| Olympic| Games| in| |202|4|.

|2|.| **|Guid|ed| Tours|**|:| Consider| taking| a| guided| tour| of| Lille|'s| historic| architecture|.| The| city| is| rich| in| history| and| offers| stunning| examples| of| architecture| from| different| eras|.

|3|.| **|Pal|ais| des| Be|aux|-Ar|ts| de| Lille|**|:| This| museum| houses| an| impressive| collection| of| paintings|,| sculptures|,| and| decorative| arts|.| It's| one| of| the| largest| fine| arts| museums| in| France|,| making| it| a| must|-|visit| for| art| lovers|.

|4|.| **|C|ul|inary| Experiences|**|:| For| those| interested| in| cooking|,| you| might| find| traditional| French| cooking| classes

In [32]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [33]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable" : {"thread_id":"abc123"}}

In [38]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage("hi, I'm an intern at EY Belgium")]},
    config,
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content="Hello! It's great to have you here. How can I assist you today? If you have any questions or need information related to your internship or anything else, please let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 267, 'total_tokens': 306, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_27322b4e16', 'id': 'chatcmpl-BFIdQiwFT2psTjbRhfFdnPkoEgR7W', 'finish_reason': 'stop', 'logprobs': None}, id='run-952c7696-0a4c-4852-addb-930a249243c8-0', usage_metadata={'input_tokens': 267, 'output_tokens': 39, 'total_tokens': 306, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
---


In [40]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage("What company might I work for in 2 yrs?")]},
    config,
):
    print(chunk)
    print(f"{'-'*80}")

{'agent': {'messages': [AIMessage(content='To provide a more tailored answer, could you share your field of study or area of interest? Additionally, knowing your career goals would help in suggesting potential companies you might work for in two years.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 323, 'total_tokens': 364, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_27322b4e16', 'id': 'chatcmpl-BFIguff21fxp52qDB2Ip0r3Af21jF', 'finish_reason': 'stop', 'logprobs': None}, id='run-b547f32b-d697-43fb-aca0-4147119dd1ff-0', usage_metadata={'input_tokens': 323, 'output_tokens': 41, 'total_tokens': 364, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin